# Import statements

In [1]:
import os
import csv
import shutil
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
import drms
import time
import glob
from sunpy.map import Map
import astropy

# Function for requesting a downloading data

In [2]:
# download aia images through drms
def download_aia_images(tref, delta_t, x_hpc, y_hpc, n_pixel,
                        path_dir='./', notify='john_doe@something.com'):
    
    client = drms.Client(verbose=False)
    
    qstr = "aia.lev1_euv_12s["+ tref +"/" + str(delta_t) + "m]"
    print(f"Data export query:\n  {qstr}\n")

    ###############################################################################
    # Construct the dictionary specifying that we want to request a cutout.
    # This is done via the ``im_patch`` command.
    
    # The ``t`` controls whether tracking is disabled (``1``) or enabled (``0``).
    # ``r`` controls the use of sub-pixel registration.
    # ``c`` controls whether off-limb pixels are filled with NaNs.
    # For additional details about ``im_patch``, 
    # see the `documentation <http://jsoc.stanford.edu/doxygen_html/group__im__patch.html>`_.
    process = {
        "aia_scale_aialev1":{           # Scale for transforming data as level 1p5
        "mpt": "aia.master_pointing3h",
        },
        "im_patch": {
            "t_ref": tref,
            "t": 1,
            "r": 0,
            "c": 0,
            "locunits": "arcsec",
            "boxunits": "pixels",
            "x": x_hpc,
            "y": y_hpc,
            "width": n_pixel,
            "height": n_pixel,
        }
    }

    
    # Submit export request using the 'fits' protocol
    print("Submitting export request...")
    result = client.export(
        qstr,
        method="url",
        protocol="fits",
        email=notify,
        process=process,
        filenamefmt="{seriesname}.{T_REC:A}.{WAVELNTH}.{segment}",
    )

    # Print request URL.
    print(f"\nRequest URL: {result.request_url}")
    print(f"{int(len(result.urls))} file(s) available for download.\n")

    # Download selected files.
    result.wait()
    result.download(path_dir)
    print("Download finished.")

# Functions for creating a saving a datacube

In [3]:
# Given a filename in the standard format from JSOC download, find out what the wavelength is
def WavelengthFromFilename(filename):
    # Position of "." after wavelength
    end = filename.find(".image")
    if end < 0:
        return -1
    
    start = end - 1
    
    while filename[start] != '.':
        start = start - 1
    
    start = start + 1
    
    return int(filename[start:end])

# Order of wavelengths from low to high temperature
wavelengthIndexDict = {304:0, 131:1, 171:2, 193:3, 211:4, 335:5, 94:6}

# And the inverse
wavelengthInvIndexDict = {v: k for k, v in wavelengthIndexDict.items()}


def save_fits_datacube(filename, aia_data_folder,array_tuple):
    
    downloaded_files = glob.glob(os.path.join(aia_data_folder, "aia*"))
    
    binnedFiles = [[], [], [], [], [], [], []]

    for file in downloaded_files:
        wav = WavelengthFromFilename(file)

        # skip if it is a spike file
        if wav == -1:
            continue

        # otherwise bin it
        binnedFiles[wavelengthIndexDict[wav]].append(file)

    # Now also sort each bin by time

    for wav_bin in binnedFiles:
        wav_bin.sort()

    out_arr = np.zeros(array_tuple, dtype=np.int16)

    for t_ind in range(array_tuple[3]):
        for e_ind in range(array_tuple[2]):
            aux = Map(binnedFiles[e_ind][t_ind]).data.T
            out_arr[:, :, e_ind, t_ind] = aux

    astropy.io.fits.writeto(filename, out_arr, overwrite=True)

# Main

In [4]:
notify = "massa.p@dima.unige.it" # change this!

# Path of the folder where to store the AIA maps and the datacube fits file
path_dir = os.path.join(".", "example") 
if not os.path.exists(path_dir):
    os.mkdir(path_dir)

# Example: flare 52, flare list 1
tref    = "2010-05-22T21:39:00" # start time of the flare
delta_t = 1    # time interval to download (starting from tref) in minutes
x_hpc   = -147 # X coordinate of the center of the cutout (in arcsec)
y_hpc   = -234 # Y coordinate of the center of the cutout (in arcsec)
n_pixel = 200  # number of pixels of the cutout (both on the x and y axis)

# Retrieve and download AIA data
download_aia_images(tref, delta_t, x_hpc, y_hpc, n_pixel, path_dir=path_dir, notify=notify)

# Save fits file of the datacube
filename = os.path.join(path_dir, "datacube.fits")
# Tuple with the shape of the datacube to save
array_tuple = [200,200,7,5]
save_fits_datacube(filename, path_dir,array_tuple)

Data export query:
  aia.lev1_euv_12s[2010-05-22T21:39:00/1m]

Submitting export request...

Request URL: http://jsoc.stanford.edu/SUM39/D1556797993/S00000
35 file(s) available for download.

Download finished.
